# 一、生成器

In [3]:
ls = [i**2 for i in range(1, 1000001)]

In [4]:
for i in ls:
    pass

缺点：占用大量内存

**生成器**  
  
  （1）采用惰性计算的方式   
     
  （2）无需一次性存储海量数据  
  
  （3）一边执行一边计算，只计算每次需要的值
  
  （4）实际上一直在执行next()操作，直到无值可取


## 01. 生成器表达式

* 海量数据，不需存储

In [ ]:
squares = (i**2 for i in range(1000000))

In [168]:
for i in squares:
    pass

* 求0~100的和

无需显示存储全部数据，节省内存

In [1]:
sum((i for i in range(101)))

5050

## 02. 生成器函数 — yield

* 生产斐波那契数列

数列前两个元素为1，1。之后的元素为其前两个元素之和

In [2]:
def fib(max):
    ls = []
    n, a, b = 0, 1, 1
    while n < max:
        ls.append(a)
        a, b = b, a + b
        n = n + 1
    return ls


fib(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

* 中间尝试

In [3]:
def fib(max):
    n, a, b = 0, 1, 1
    while n < max:
        print(a)
        a, b = b, a + b
        n = n + 1


fib(10)

1
1
2
3
5
8
13
21
34
55


* 构造生成器函数

在每次调用next()的时候执行，遇到yield语句返回，再次执行时从上次返回的yield语句处继续执行

In [4]:
def fib(max):
    n, a, b = 0, 1, 1
    while n < max:
        yield a
        a, b = b, a + b
        n = n + 1
        

fib(10)

<generator object fib at 0x00000213EB4514C0>

In [5]:
for i in fib(10):
    print(i)

1
1
2
3
5
8
13
21
34
55


# 二、迭代器

## 2.1 可迭代对象

可直接作用于for循环的对象统称为可迭代对象：Iterable

**（1）列表、元组、字符串、字典、集合、文件**

可以使用isinstance()判断一个对象是否是Iterable对象

In [6]:
from collections import Iterable

isinstance([1, 2, 3], Iterable)

True

In [7]:
isinstance({"name": "Sarah"}, Iterable)

True

In [8]:
isinstance('Python', Iterable)

True

**（2）生成器**

In [9]:
squares = (i**2 for i in range(5))
isinstance(squares, Iterable)

True

生成器不但可以用于for循环，还可以被next()函数调用

In [10]:
print(next(squares))
print(next(squares))
print(next(squares))
print(next(squares))
print(next(squares))

0
1
4
9
16


直到没有数据可取，抛出StopIteration

In [11]:
print(next(squares))

StopIteration: 

**可以被next()函数调用并不断返回下一个值，直至没有数据可取的对象称为迭代器：Iterator**

## 2.2 迭代器

可以使用isinstance()判断一个对象是否是Iterator对象

**（1） 生成器都是迭代器**

In [12]:
from collections import Iterator

squares = (i**2 for i in range(5))
isinstance(squares, Iterator)

True

**（2） 列表、元组、字符串、字典、集合不是迭代器**

In [13]:
isinstance([1, 2, 3], Iterator)

False

可以通过iter(Iterable)创建迭代器

In [14]:
isinstance(iter([1, 2, 3]), Iterator)

True

for item in Iterable 等价于：
  
    先通过iter()函数获取可迭代对象Iterable的迭代器
  
    然后对获取到的迭代器不断调用next()方法来获取下一个值并将其赋值给item
   
    当遇到StopIteration的异常后循环结束

**（3）zip enumerate 等itertools里的函数是迭代器**

In [15]:
x = [1, 2]
y = ["a", "b"]
zip(x, y)

In [16]:
for i in zip(x, y):
    print(i)
    
isinstance(zip(x, y), Iterator)

(1, 'a')
(2, 'b')


True

In [17]:
numbers = [1, 2, 3, 4, 5]
enumerate(numbers)

In [18]:
for i in enumerate(numbers):
    print(i)
    
isinstance(enumerate(numbers), Iterator)

(0, 1)
(1, 2)
(2, 3)
(3, 4)
(4, 5)


True

**(4) 文件是迭代器**

In [20]:
with open("测试文件.txt", "w", encoding = "utf-8") as f:
    print(isinstance(f, Iterator))

True


**（5）迭代器是可耗尽的**

In [21]:
squares = (i**2 for i in range(5))
for square in squares:
    print(square)

0
1
4
9
16


In [22]:
for square in squares:
    print(square)

**（6）range()不是迭代器**

In [23]:
numbers = range(10)
isinstance(numbers, Iterator)

False

In [24]:
print(len(numbers))   # 有长度
print(numbers[0])     # 可索引
print(9 in numbers)   # 可存在计算
next(numbers)         # 不可被next()调用

10
0
True


TypeError: 'range' object is not an iterator

In [25]:
for number in numbers:
    print(number)

0
1
2
3
4
5
6
7
8
9


In [78]:
for number in numbers:
    print(number)

0
1
2
3
4
5
6
7
8
9


由上面两个循环看出，range()不会被耗尽

可以称range()为懒序列  
  
    它是一种序列
      
    但并不包含任何内存中的内容
      
    而是通过计算来回答问题

# 三、装饰器

## 01. 需求的提出

（1）需要对已开发上线的程序添加某些功能
    
（2）不能对程序中函数的源代码进行修改
  
（3）不能改变程序中函数的调用方式

**比如说，要统计每个函数的运行时间**

In [ ]:
def f1():
    pass


def f2():
    pass


def f3():
    pass

f1()
f2()
f3()

**没问题，我们有装饰器！！！**

## 02. 函数对象

函数是Python中的第一类对象

（1）可以把函数赋值给变量  
  
（2）对该变量进行调用，可实现原函数的功能

In [27]:
def square(x):
    return x**2

print(type(square))      # square 是function类的一个实例

<class 'function'>


In [28]:
pow_2 = square          # 可以理解成给这个函数起了个别名pow_2
print(pow_2(5))
print(square(5))

25
25


> 可以将函数作为参数进行传递

## 03. 高阶函数

（1）接收函数作为参数  
  
（2）或者返回一个函数  
  
   **满足上述条件之一的函数称之为高阶函数**

In [29]:
def square(x):
    return x**2


def pow_2(fun):
    return fun


f = pow_2(square)
f(8)

64

In [30]:
print(f == square)

True


## 04. 嵌套函数

**在函数内部定义一个函数**

In [31]:
def outer():
    print("outer is running")
    
    def inner():
        print("inner is running")
        
    inner()


outer()

outer is running
inner is running


## 05. 闭包

In [32]:
def outer():
    x = 1
    z = 10
    
    def inner():
        y = x+100
        return y, z
        
    return inner


f = outer()                # 实际上f包含了inner函数本身+outer函数的环境
print(f)

<function outer.<locals>.inner at 0x00000213EB4FDF28>


In [33]:
print(f.__closure__)         # __closure__属性中包含了来自外部函数的信息
for i in f.__closure__:
    print(i.cell_contents)

(<cell at 0x00000213EB460D08: int object at 0x0000000077FB6C40>, <cell at 0x00000213EB460B88: int object at 0x0000000077FB6D60>)
1
10


In [34]:
res = f()
print(res)

(101, 10)


**闭包：延伸了作用域的函数**  
  
**如果一个函数定义在另一个函数的作用域内，并且引用了外层函数的变量，则该函数称为闭包**
  
**闭包是由函数及其相关的引用环境组合而成的实体(即：闭包=函数+引用环境)**

* 一旦在内层函数重新定义了相同名字的变量，则变量成为局部变量

In [35]:
def outer():
    x = 1
    
    def inner():
        x = x+100
        return x
        
    return inner


f = outer()             
f()

UnboundLocalError: local variable 'x' referenced before assignment

nonlocal允许内嵌的函数来修改闭包变量

In [36]:
def outer():
    x = 1
    
    def inner():
        nonlocal x
        x = x+100
        return x  
    return inner


f = outer()             
f()

101

## 06. 一个简单的装饰器

**嵌套函数实现**

In [37]:
import time

def timer(func):
    
    def inner():
        print("inner run")
        start = time.time()
        func()
        end = time.time()
        print("{} 函数运行用时{:.2f}秒".format(func.__name__, (end-start)))
    
    return inner


def f1():
    print("f1 run")
    time.sleep(1)



f1 = timer(f1)             # 包含inner()和timer的环境，如传递过来的参数func
f1()

inner run
f1 run
f1 函数运行用时1.01秒


**语法糖**

In [38]:
import time

def timer(func):
    
    def inner():
        print("inner run")
        start = time.time()
        func()
        end = time.time()
        print("{} 函数运行用时{:.2f}秒".format(func.__name__, (end-start)))
    
    return inner

@timer                      # 相当于实现了f1 = timer(f1)
def f1():
    print("f1 run")
    time.sleep(1)
    
    
f1()

inner run
f1 run
f1 函数运行用时1.01秒


## 07. 装饰有参函数

In [91]:
import time


def timer(func):
    
    def inner(*args, **kwargs):
        print("inner run")
        start = time.time()
        func(*args, **kwargs)
        end = time.time()
        print("{} 函数运行用时{:.2f}秒".format(func.__name__, (end-start)))
    
    return inner


@timer                # 相当于实现了f1 = timer(f1)
def f1(n):
    print("f1 run")
    time.sleep(n)

    
f1(2)

inner run
f1 run
f1 函数运行用时2.00秒


被装饰函数有返回值的情况

In [92]:
import time


def timer(func):
    
    def inner(*args, **kwargs):
        print("inner run")
        start = time.time()
        res = func(*args, **kwargs)
        end = time.time()
        print("{} 函数运行用时{:.2f}秒".format(func.__name__, (end-start)))
        return res
    
    return inner


@timer                   # 相当于实现了f1 = timer(f1)
def f1(n):
    print("f1 run")
    time.sleep(n)
    return "wake up"
    
res = f1(2)
print(res)

inner run
f1 run
f1 函数运行用时2.00秒
wake up


## 08. 带参数的装饰器

装饰器本身要传递一些额外参数

* 需求：有时需要统计绝对时间，有时需要统计绝对时间的2倍

In [95]:
def timer(method):
    
    def outer(func):
    
        def inner(*args, **kwargs):
            print("inner run")
            if method == "origin":
                print("origin_inner run")
                start = time.time()
                res = func(*args, **kwargs)
                end = time.time()
                print("{} 函数运行用时{:.2f}秒".format(func.__name__, (end-start)))
            elif method == "double":
                print("double_inner run")
                start = time.time()
                res = func(*args, **kwargs)
                end = time.time()
                print("{} 函数运行双倍用时{:.2f}秒".format(func.__name__, 2*(end-start)))
            return res
    
        return inner
    
    return outer


@timer(method="origin")  # 相当于timer = timer(method = "origin")   f1 = timer(f1)
def f1():
    print("f1 run")
    time.sleep(1)
    
    
@timer(method="double")
def f2():
    print("f2 run")
    time.sleep(1)


f1()
print()
f2()

inner run
origin_inner run
f1 run
f1 函数运行用时1.00秒

inner run
double_inner run
f2 run
f2 函数运行双倍用时2.00秒


**理解闭包是关键！！！**

## 09. 何时执行装饰器

* 一装饰就执行，不必等调用

In [39]:
func_names=[]
def find_function(func):
    print("run")
    func_names.append(func)
    return func


@find_function
def f1():
    print("f1 run")
    

@find_function
def f2():
    print("f2 run")
    

@find_function
def f3():
    print("f3 run")
    


run
run
run


In [40]:
for func in func_names:
    print(func.__name__)
    func()
    print()

f1
f1 run

f2
f2 run

f3
f3 run



## 10. 回归本源

* 原函数的属性被掩盖了

In [41]:
import time

def timer(func):
    def inner():
        print("inner run")
        start = time.time()
        func()
        end = time.time()
        print("{} 函数运行用时{:.2f}秒".format(func.__name__, (end-start)))
    
    return inner

@timer                # 相当于实现了f1 = timer(f1)
def f1():
    time.sleep(1)
    print("f1 run")

print(f1.__name__)    

inner


* 回来

In [42]:
import time
from functools import wraps


def timer(func):
    @wraps(func)
    def inner():
        print("inner run")
        start = time.time()
        func()
        end = time.time()
        print("{} 函数运行用时{:.2f}秒".format(func.__name__, (end-start)))
    
    return inner

@timer                # 相当于实现了f1 = timer(f1)
def f1():
    time.sleep(1)
    print("f1 run")

print(f1.__name__) 
f1()

f1
inner run
f1 run
f1 函数运行用时1.00秒
